<a href="https://colab.research.google.com/github/SasamiScott/Video-Games-and-Player-Retention/blob/master/HDSI_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pickle
import numpy as np
import urllib.request
import time
import pandas as pd
import json
import re

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Gathering User Data

In [0]:
#This contains code and data from McAuley 
user_f = '/content/gdrive/My Drive/Colab Notebooks/user_id_lookup.txt'
users = pickle.load(open(user_f,'rb'))

#From his user file I want every userId on record, but this are mixed with usernames
user_ids = []
for id in range(len(users)):
  try:
    int(users[id]) #ids are all numbers
    if len(users[id]) == 17 and users[id][0] == '7': # ids are 17 chracters and begin with 7
      user_ids.append(users[id])
  except:
    continue
  
print("Number of User Ids: " + str(len(user_ids)))

Number of User Ids: 14445


In [0]:
#Set ups the non changing components of the url I'm using to test userIds and get user data on games owned
u_base = 'http://api.steampowered.com/IPlayerService'
u_db = 'GetOwnedGames/v0001'
u_key = '?key='
key = 'A43433F6225B90C91D6DD1F3783AF943'
u_format = 'format=json'
u_id = 'steamid='

public_users = [] #Ids
public_user_data = [] #Games owned and associated playtimes

In [0]:
# PERSONAL: check 13000 - 14445 again only a 22 user id difference from 12000 - 13000

for i in range(1, len(user_ids)): # INCREMENT RANGE TO 1000
  user_x = user_ids[i]
  steamids = user_x # the user id is the only url segment that changes
  url = u_base + '/' + u_db + '/' + '?' + u_key + key + '&' + u_id + steamids + '&' + u_format
  with urllib.request.urlopen(url) as response:
    xml = response.read()
    if len(xml) > 15: # data will be over 15 chracters long if the user's settings are set to public
      public_users.append(user_x)
      public_user_data.append(xml)
    time.sleep(1) # Add a second for each iteration to avoid SteamApi data limits

In [0]:
bad_users = [5948, 6924, 9755] #indexes of weird ids that go beyond my search parameters (i.e. deleted or dumb user names)

'76569696969696961'

In [0]:
# Turns the jsons in to dicts to panda datrames and combines them
# 1st public user is the base dictionary 
public_user_dict = json.loads(public_user_data[0][12:-1]) # 0-12 and -1 are indexes that hold response formatting
public_user_dict['userId'] = public_users[0] # add userId for future reference
user_df = pd.DataFrame([public_user_dict], columns=public_user_dict.keys())

# Same process for all other users
for j in range(1, len(public_user_data)):
  userx_data = json.loads(public_user_data[j][12:-1])
  userx_data['userId'] = public_users[j]
  user_df = user_df.append(userx_data, ignore_index=True)

In [0]:
#processed users were initialled stored in 2 dataframes
#user_df.to_csv(r'/content/gdrive/My Drive/Colab Notebooks/user_df2.csv')
#user_df2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/user_df.csv') 

In [0]:
#Combined and saved table with new indexes
#public_user_table = user_df2.append(user_df, sort=False).reset_index().drop(['index', 'Unnamed: 0'], axis=1)
#public_user_table.to_csv(r'/content/gdrive/My Drive/Colab Notebooks/public_user_table.csv')

# Organizing and Gathering Game Data

In [0]:
public_user_table = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/public_user_table.csv')
public_user_table

,Unnamed: 0,game_count,games,userId
0,0,88,"[{'appid': 4000, 'playtime_forever': 281, 'pla...",76561198156664158
1,1,106,"[{'appid': 240, 'playtime_forever': 0, 'playti...",76561198043472122
2,2,211,"[{'appid': 10, 'playtime_forever': 25, 'playti...",76561198048353577
3,3,185,"[{'appid': 10, 'playtime_forever': 118, 'playt...",76561198100701221
4,4,152,"[{'appid': 300, 'playtime_2weeks': 13, 'playti...",76561198078738103
...,...,...,...,...
1899,1899,63,"[{'appid': 220, 'playtime_forever': 615, 'play...",76561198081816341
1900,1900,109,"[{'appid': 4000, 'playtime_forever': 12834, 'p...",76561198083190937
1901,1901,22,"[{'appid': 12200, 'playtime_forever': 1719, 'p...",76561198083454793
1902,1902,149,"[{'appid': 10, 'playtime_forever': 0, 'playtim...",76561198083857294


In [0]:
str_data = public_user_table.iloc[0]['games']
game_num = re.findall(r':* +[0-9]*[,]', str_data)
game_num

[': 4000,',
 ': 281,',
 ': 0,',
 ': 0,',
 ': 17390,',
 ': 1217,',
 ': 0,',
 ': 0,',
 ': 33910,',
 ': 162,',
 ': 0,',
 ': 0,',
 ': 33930,',
 ': 226,',
 ': 0,',
 ': 0,',
 ': 219540,',
 ': 0,',
 ': 0,',
 ': 0,',
 ': 400,',
 ': 196,',
 ': 0,',
 ': 0,',
 ': 620,',
 ': 1091,',
 ': 0,',
 ': 0,',
 ': 105600,',
 ': 21107,',
 ': 3294,',
 ': 0,',
 ': 104900,',
 ': 134,',
 ': 0,',
 ': 0,',
 ': 227280,',
 ': 315,',
 ': 0,',
 ': 0,',
 ': 368730,',
 ': 27,',
 ': 0,',
 ': 0,',
 ': 230410,',
 ': 55811,',
 ': 0,',
 ': 0,',
 ': 34870,',
 ': 231,',
 ': 0,',
 ': 0,',
 ': 8870,',
 ': 943,',
 ': 0,',
 ': 0,',
 ': 242760,',
 ': 3260,',
 ': 0,',
 ': 0,',
 ': 248820,',
 ': 193,',
 ': 0,',
 ': 0,',
 ': 218620,',
 ': 7916,',
 ': 99,',
 ': 0,',
 ': 252490,',
 ': 231,',
 ': 0,',
 ': 0,',
 ': 700580,',
 ': 0,',
 ': 0,',
 ': 0,',
 ': 252610,',
 ': 385,',
 ': 0,',
 ': 0,',
 ': 252950,',
 ': 22,',
 ': 49733,',
 ': 2654,',
 ': 0,',
 ': 107410,',
 ': 155,',
 ': 0,',
 ': 0,',
 ': 257850,',
 ': 688,',
 ': 0,',
 ': 0,',
 ':

In [0]:
str_data

"[{'appid': 4000, 'playtime_forever': 281, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 17390, 'playtime_forever': 1217, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 33910, 'playtime_forever': 162, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 33930, 'playtime_forever': 226, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 219540, 'playtime_forever': 0, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 400, 'playtime_forever': 196, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 620, 'playtime_forever': 1091, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}, {'appid': 105600, 'playtime_forever': 21107, 'playtime_windows_forever': 3294, 'playtime_mac